In [1]:
import os
import json
import time

from datetime import datetime

from tqdm.notebook import tqdm

from react_agent.src.util.llm_proxy import LLM_PROXY
from react_agent.src.util.tools_fabric import ToolsFabric

from react_agent.src.config.system_parameters import TriageSettings

from experiments.models.experiment_models import LabeledQAPairFacts, ExperimentResult
from experiments.metrics.fact_score.fact_scorer import FactScorer
from experiments.metrics.bert_score import BertScore
from experiments.metrics.agent_judge import AgentJudgeEvaluator

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
AGENT_JUDGE_MODEL = "anthropic--claude-3.5-sonnet"

fact_scorer = FactScorer()
path_to_source_data = "./resources/02_facts/"

date_str = datetime.now().strftime("%d%m%Y")
path_to_results = f"./results/{date_str}"

if not os.path.exists(path_to_results):
    os.makedirs(path_to_results)

time_str = datetime.now().strftime("%H%M%S")

result_file_name = f"{path_to_results}/{time_str}_results.json"
failed_experiments_file_name = f"{path_to_results}/{time_str}_failed_experiments.json"

In [4]:
file_path = os.path.abspath(path_to_source_data + "data_set_facts.json")

with open(file_path, encoding="utf8") as f:
    data = json.load(f)
    data_set = [LabeledQAPairFacts(**item) for item in data]

In [5]:
data_set_for_experiment = data_set

In [6]:
async def run_experiment(data_row):
    result = ExperimentResult(
        id=data_row.id,
        question=data_row.question,
        answer=data_row.answer,
        product=data_row.product,
        category=data_row.category,
        persona=data_row.persona,
        activity=data_row.activity,
        country=data_row.country,
    )
    print(f"Processing question with id: {data_row.id}")

    tools = ToolsFabric.get_tools_for_category(
        use_mcp=False,
        configuration=TriageSettings.Categories.ALL,
    )

    print("\tAsking LLM question...")

    run_start_time = time.perf_counter()
    llm_response = LLM_PROXY.invoke(input=data_row.question)
    run_end_time = time.perf_counter()

    result.tools_used = None
    result.tool_calls_count = 0
    result.excecution_time_seconds = run_end_time - run_start_time
    result.model_used = LLM_PROXY.get_used_model()
    result.tokens_consumed = LLM_PROXY.get_token_usage()
    result.llm_call_count = LLM_PROXY.get_call_count()
    result.facts = data_row.facts
    result.generated_answer = llm_response

    print("\tFinished agent execution")

    print("\tCalculating FactScore...")
    result.fact_score = await fact_scorer.get_fact_score(
        facts=data_row.facts,
        knowledge_source=result.generated_answer,
    )

    print("\tCalculating BERTScore...")
    result.bert_score = BertScore.compute_score(
        expected_response=data_row.answer, actual_response=result.generated_answer
    )

    print("\tEvaluating agent response with Agent as a Judge...")
    result.agent_judge_model = AGENT_JUDGE_MODEL
    agent_evaluator = AgentJudgeEvaluator(model=AGENT_JUDGE_MODEL)

    try:
        agent_judge_outcome = agent_evaluator.evaluate(
            question=data_row.question,
            expert_answer=data_row.answer,
            generated_answer=result.generated_answer,
        )
    except Exception as e:
        print(f"\tAn error occurred during agent evaluation: {e}")
        print("\t Trying to run evaluation again...")
        agent_judge_outcome = agent_evaluator.evaluate(
            question=data_row.question,
            expert_answer=data_row.answer,
            generated_answer=result.generated_answer,
        )

    result.agent_judge_outcome = agent_judge_outcome.answer
    result.agent_judge_reasoning = agent_judge_outcome.reasoning

    result.agent_judge_call_count = agent_evaluator.get_llm_call_count()
    result.agent_judge_tokens_consumed = agent_evaluator.get_token_consumption()

    print(f"\tFinished experiment for id: {data_row.id}\n")

    return result

In [7]:
experiment_results: list[ExperimentResult] = []

In [10]:
for data_row in tqdm(data_set_for_experiment, desc="Running cases", unit="case"):
    if data_row.id in [result.id for result in experiment_results]:
        print(f"Skipping already processed id: {data_row.id}")
        continue
    try:
        result = await run_experiment(data_row)

        experiment_results.append(result)
    except Exception as e:
        print(f"Error processing id {data_row.id}: {e}")

Running cases:   0%|          | 0/91 [00:00<?, ?case/s]

Skipping already processed id: EA-1
Skipping already processed id: EA-2
Skipping already processed id: EA-3
Skipping already processed id: EA-4
Skipping already processed id: EA-5
Skipping already processed id: EA-6
Skipping already processed id: EA-7
Skipping already processed id: EA-8
Skipping already processed id: EA-9
Skipping already processed id: EA-10
Skipping already processed id: EA-11
Skipping already processed id: EA-12
Skipping already processed id: EA-13
Skipping already processed id: EA-14
Skipping already processed id: EA-15
Skipping already processed id: EA-16
Skipping already processed id: EA-17
Skipping already processed id: EA-18
Skipping already processed id: EA-19
Skipping already processed id: EA-20
Skipping already processed id: EA-21
Skipping already processed id: EA-22
Skipping already processed id: EA-23
Skipping already processed id: EA-24
Skipping already processed id: EA-25
Skipping already processed id: EA-26
Skipping already processed id: EA-27
Skipping a

In [11]:
# Serialize experiment results to list of dicts
import pandas as pd

records = [r.model_dump(mode="json") for r in experiment_results]
# Make it a dataframe in order to flatten nested structures
df = pd.json_normalize(records)

df

,id,question,answer,product,category,persona,activity,country,facts,bert_score,...,fact_score.supporting_fact_score,fact_score.combined_fact_score,fact_score.direct_facts,fact_score.supporting_facts,agent_judge_tokens_consumed.input_tokens,agent_judge_tokens_consumed.output_tokens,agent_judge_tokens_consumed.total_tokens,tokens_consumed.input_tokens,tokens_consumed.output_tokens,tokens_consumed.total_tokens
0,EA-1,How can I attach additional files to standard ...,To attach additional files to the standard XML...,ERP (only SAP S/4HANA Cloud Public Edition),How to Develop (Maintenance - Customer case su...,Developer (Customer side),Custom Development,Cross,[{'fact': 'You can attach additional files to ...,0.085529,...,0.000000,0.000000,[{'fact': 'You can attach additional files to ...,[{'fact': 'The enhancement is implemented in A...,6655,3,6658,19,2299,2318
1,EA-2,"For a Greece customer in Public Cloud, what ar...",To enhance the generated XML file for a Greece...,ERP (only SAP S/4HANA Cloud Public Edition),How to Develop (Maintenance - Customer case su...,Developer (Customer side),Custom Development,GR,[{'fact': 'To enhance the generated XML for a ...,0.161259,...,0.333333,0.333333,[{'fact': 'To enhance the generated XML for a ...,[{'fact': 'You can optionally create a BAdI fi...,6253,3,6256,34,2671,2705
2,EA-3,For a Greece customer invoice process in Publi...,"Yes, you can enhance or extend the generated X...",ERP (only SAP S/4HANA Cloud Public Edition),How to Develop (Maintenance - Customer case su...,Developer (Customer side),Custom Development,GR,[{'fact': 'You can enhance or extend the gener...,0.198058,...,0.000000,0.000000,[{'fact': 'You can enhance or extend the gener...,[{'fact': 'The method SET_OUTPUT_DATA belongs ...,5942,3,5945,32,1890,1922
3,EA-4,"As a Public Cloud customer in Spain, can I ext...","Yes, as a Public Cloud customer in Spain, you ...",ERP (only SAP S/4HANA Cloud Public Edition),Documentation (Maintenance - Customer case sup...,Developer (Customer side),Custom Development,ES,[{'fact': 'A Public Cloud customer in Spain ca...,0.190952,...,0.000000,0.000000,[{'fact': 'A Public Cloud customer in Spain ca...,[{'fact': 'It is possible to define that for s...,5672,3,5675,26,1550,1576
4,EA-5,"As a Public Cloud customer in Italy, can I ext...","Yes, as a Public Cloud customer in Italy, you...",ERP (only SAP S/4HANA Cloud Public Edition),Documentation (Maintenance - Customer case sup...,Developer (Customer side),Custom Development,IT,[{'fact': 'As a Public Cloud customer in Italy...,0.155903,...,0.000000,0.000000,[{'fact': 'As a Public Cloud customer in Italy...,[{'fact': 'You can define that for some types ...,6829,3,6832,26,827,853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,EA-19,For a Turkish customer invoice process in Publ...,"Yes, you can enhance the generated XML file fo...",ERP (only SAP S/4HANA Cloud Public Edition),Documentation (Maintenance - Customer case sup...,Developer (Customer side),Custom Development,TR,[{'fact': 'You can enhance the generated XML f...,0.152290,...,0.000000,0.000000,[{'fact': 'You can enhance the generated XML f...,[{'fact': 'Enhancing the XML file allows you t...,6212,3,6215,33,1082,1115
87,EA-28,"In Public, what are the prerequisites for crea...","In Public cloud, the prerequisites to creating...",ERP (only SAP S/4HANA Cloud Public Edition),Documentation (Maintenance - Customer case sup...,Developer (Customer side),Custom Development,Cross,"[{'fact': 'In Public cloud, the country/region...",0.124619,...,0.000000,0.000000,"[{'fact': 'In Public cloud, the country/region...",[],4592,3,4595,24,2308,2332
88,EA-45,Before implementing a new process in DRC using...,"Before you implement a new process, there are ...",ERP,How to Configure (Maintenance - Customer case ...,Functional consultant (Customer side),Design,Cross,[{'fact': 'You must determine which new action...,0.222315,...,0.000000,0.000000,[{'fact': 'You must determine which n

In [12]:
# Convert to list of dictionaries (JSON objects)
json_list = df.to_dict(orient="records")

with open(result_file_name, "w") as f:
    json.dump(json_list, f, indent=2)